In [1]:
import numpy as np
import tensorflow as tf
import os
import config

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

print(f"Weather classes for classification:\n{config.CLASS_NAMES}")

['.amlignore',
 '.amlignore.amltmp',
 'cloudy',
 'dew',
 'fogsmog',
 'frost',
 'glaze',
 'hail',
 'lightning',
 'rain',
 'rainbow',
 'rime',
 'sandstorm',
 'shine',
 'snow',
 'sunrise']

In [2]:
train_data = tf.keras.utils.image_dataset_from_directory(
  config.DATA_DIR,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(config.IMG_HEIGHT, config.IMG_WIDTH),
  batch_size=config.BATCH_SIZE)

val_data = tf.keras.utils.image_dataset_from_directory(
  config.DATA_DIR,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(config.IMG_HEIGHT, config.IMG_WIDTH),
  batch_size=config.BATCH_SIZE)

test_size = int(len(val_data) / 2)
test_data = val_data.take(test_size)
val_data = val_data.skip(test_size)

Found 7984 files belonging to 14 classes.
Using 6388 files for training.
Found 7984 files belonging to 14 classes.
Using 1596 files for validation.


In [3]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size=AUTOTUNE)

In [4]:
from tensorflow.keras import layers, Sequential, optimizers

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1)
])

In [5]:
def model_builder(hp):
  
  model = Sequential([
    layers.Rescaling(1./255, input_shape=(config.IMG_HEIGHT, config.IMG_WIDTH, 3)),
    data_augmentation,
    layers.Conv2D(hp.Int("conv1", min_value=32, max_value=96, step=32), 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(hp.Int("conv2", min_value=64, max_value=128, step=32), 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(hp.Int("conv3", min_value=64, max_value=128, step=32), 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(hp.Int("dense_units", min_value=128, max_value=768, step=128), activation='relu'),
    layers.Dense(config.NUM_CLASSES)
  ])

  lr = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3, 1e-4])
  opt = optimizers.Adam(learning_rate=lr)

  model.compile(optimizer=opt,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
	monitor="val_loss",
	patience=config.EARLY_STOPPING_PATIENCE,
	restore_best_weights=True)

In [7]:
import sys
!{sys.executable} -m pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 2.1 MB/s eta 0:00:0000:01


In [8]:
import keras_tuner as kt

epochs = 15

tuner = kt.Hyperband(
  model_builder,
  objective="val_accuracy",
  max_epochs=epochs,
  factor=3,
  seed=42,
  directory="./tuner-logs",
  project_name="hyperband"
)

In [14]:
tuner.search(
	x=train_data,
	validation_data=val_data,
	batch_size=config.BATCH_SIZE,
	callbacks=[early_stop],
	epochs=epochs
)

bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Optimal number of filters in conv_1 layer: {bestHP.get('conv1')}")
print(f"Optimal number of filters in conv_2 layer: {bestHP.get('conv2')}")
print(f"Optimal number of filters in conv_2 layer: {bestHP.get('conv3')}")
print(f"Optimal number of units in dense layer: {bestHP.get('dense_units')}")
print(f"Optimal learning rate: {bestHP.get('learning_rate'):.4f}")

Optimal number of filters in conv_1 layer: 64
Optimal number of filters in conv_2 layer: 96
Optimal number of filters in conv_2 layer: 64
Optimal number of units in dense layer: 640
Optimal learning rate: 0.0001


In [15]:
hypermodel = tuner.hypermodel.build(bestHP)
hypermodel.fit(train_data, validation_data=val_data, batch_size=config.BATCH_SIZE, epochs=config.EPOCHS, callbacks=[early_stop])

Epoch 1/15
200/200 [==============================] - 28s 136ms/step - loss: 1.6375 - accuracy: 0.4635 - val_loss: 1.4646 - val_accuracy: 0.4874
Epoch 2/15
200/200 [==============================] - 27s 136ms/step - loss: 1.2201 - accuracy: 0.5975 - val_loss: 1.2852 - val_accuracy: 0.5616
Epoch 3/15
200/200 [==============================] - 27s 137ms/step - loss: 1.0950 - accuracy: 0.6334 - val_loss: 1.2364 - val_accuracy: 0.5678
Epoch 4/15
200/200 [==============================] - 28s 138ms/step - loss: 1.0203 - accuracy: 0.6583 - val_loss: 1.1871 - val_accuracy: 0.5992
Epoch 5/15
200/200 [==============================] - 28s 138ms/step - loss: 0.9698 - accuracy: 0.6758 - val_loss: 1.1492 - val_accuracy: 0.6131
Epoch 6/15
200/200 [==============================] - 28s 138ms/step - loss: 0.9281 - ac

In [16]:
hypermodel.save('./models/best-hypeparams-model/')

In [19]:
hypermodel.save_weights('model-weights.h5')
with open('model-architecture.json', 'w') as json_file:
    json_file.write(hypermodel.to_json())